In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from tqdm import tqdm


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

In [40]:
model= models.vgg19(pretrained=True).features



In [41]:
CONTENT= 'content.jpg'
STYLE= 'style.jpg'



In [42]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

def image_loader(img_path):
    image= PIL.Image.open(img_path)
    loader= transforms.Compose([
        transforms.Resize((512,512)),
        transforms.ToTensor()
    ])

    image= loader(image).unsqueeze(0)
    return image.to(device, torch.float)

content_img= image_loader(CONTENT)
style_img= image_loader(STYLE)

generated= content_img.clone().requires_grad_(True).to(device).requires_grad_(True)



In [43]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19,self).__init__()
        self.req_feat=['0','5','10','19','28']
        self.model= models.vgg19(pretrained=True).features[:29]

    def forward(self, x):
        features= []
        for name, layer in enumerate(self.model):
            x= layer(x)
            if name in self.req_feat:
                features.append(x)
        return features

class Losses:
    def __init__(self, content_img, style_img, generated):
        self.content_img= content_img
        self.style_img= style_img
        self.generated= generated

    def content_loss(self):
        content_loss= F.mse_loss(self.generated, self.content_img)
        return content_loss

    def style_loss(self):
        style_loss= 0
        for layer in range(len(self.style_img)):
            style_loss+= F.mse_loss(self.generated[layer], self.style_img[layer])
        return style_loss

    def total_loss(self):
        content_loss= self.content_loss()
        style_loss= self.style_loss()
        total_loss= content_loss + style_loss
        return total_loss
        


In [44]:
model= VGG19()
epoch=7000
lr=4e-3
alpha=8
beta=70
loss= Losses(content_img, style_img, generated)
optima= optim.Adam([generated],lr=lr)

for e in tqdm(range(epoch)):
   
#extracting the features of generated, content and the original required for calculating the loss
    gen_features=model(generated)
    orig_feautes=model(content_img)
    style_featues=model(style_img)
    
    #iterating over the activation of each layer and calculate the loss and add it to the content and the style loss
    total_loss=loss.total_loss()
    #optimize the pixel values of the generated image and backpropagate the loss
    optima.zero_grad()
    total_loss.backward()
        
    optima.step()

    #print the image and save it after each 100 epoch
    if(e%700==0):
        print(total_loss)
        
        save_image(generated,"gen.png")  

  0%|          | 0/7000 [00:00<?, ?it/s]

tensor(0.1850, grad_fn=<AddBackward0>)


  0%|          | 6/7000 [01:28<28:30:05, 14.67s/it]